In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.preprocessing import OneHotEncoder

In [2]:
submission_ex = pd.read_csv('Dataset/sample_submission.csv')
submission_ex.head()

,Employee_ID,Attrition_rate
0,TID_1,0.1588
1,TID_2,0.1424
2,TID_3,0.4758
3,TID_4,0.0846
4,TID_5,0.2388


In [106]:
train = pd.read_csv('Dataset/train.csv')
test = pd.read_csv('Dataset/test.csv')
train.head()
# Goal: Make a regression model to predict the attrition rate 

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,...,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [107]:
test.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,EID_22713,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,...,4.0,type2,1.0,3,-0.9612,-0.4537,2.0,1,8,4
1,EID_9658,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,...,1.0,type2,1.0,4,-0.9612,0.7075,1.0,2,8,2
2,EID_22203,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,...,8.0,type3,1.0,4,-0.1048,0.7075,2.0,1,9,3
3,EID_7652,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,...,2.0,type0,4.0,3,-0.1048,0.7075,2.0,2,8,3
4,EID_6516,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,...,2.0,type2,4.0,4,1.6081,0.7075,2.0,2,7,4


In [126]:
# Convert nominal data to one hot encoding
towns = train['Hometown'].unique()
gender = train['Gender'].unique()
rel = train['Relationship_Status'].unique()
dsp = train['Decision_skill_possess'].unique()
unit = train['Unit'].unique()

towns.shape

IndexError: too many indices for array

In [25]:
# Make all of the data numerical - taken from https://pythonprogramming.net/working-with-non-numerical-data-machine-learning-tutorial/
def handle_non_numerical_data(df):
    columns = df.columns.values
    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents)
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1

            df[column] = list(map(convert_to_int, df[column]))

    return df

oneHot = OneHotEncoder(dtype=np.int, sparse=True)
nominals = pd.DataFrame(
    onehot.fit_transform(X[['Gender']])\
    .toarray(),
    columns=['F', 'M', 'AB', 'B+','O+', 'O-'])
nominals['edu_level'] = X.edu_level

df = handle_non_numerical_data(train)
df.drop(['Employee_ID'], axis=1, inplace=True)
df.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,1,42.0,4,0,1,10,2,4.0,4,33,...,2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,0,24.0,3,1,2,7,1,5.0,4,36,...,2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,1,58.0,3,0,0,1,2,27.0,3,51,...,2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,1,26.0,3,1,4,6,3,4.0,3,56,...,2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,1,31.0,1,0,2,7,2,5.0,4,62,...,3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827


In [71]:
# Fill NA values with averages
def fill_na(df):
    mean = df.mean(axis=0)
    df.fillna(value=mean, inplace=True)
    
    return df

df = fill_na(df)
df.head()

,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,growth_rate,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,1,42.0,4,0,1,10,2,4.0,4,33,...,2,3.0,4,0.7516,1.8688,2.000000,4,5,3,0.1841
1,0,24.0,3,1,2,7,1,5.0,4,36,...,2,4.0,3,-0.9612,-0.4537,2.000000,3,5,3,0.0670
2,1,58.0,3,0,0,1,2,27.0,3,51,...,2,1.0,4,-0.9612,-0.4537,3.000000,3,8,3,0.0851
3,1,26.0,3,1,4,6,3,4.0,3,56,...,2,1.0,3,-1.8176,-0.4537,1.891078,3,7,3,0.0668
4,1,31.0,1,0,2,7,2,5.0,4,62,...,3,3.0,1,0.7516,-0.4537,2.000000,2,8,2,0.1827


In [105]:
# Seperate into independent and dependent variable
X = df.iloc[:,:-1]
y = df.iloc[:,-1].to_numpy().reshape(1,-1)

sc_X = StandardScaler()
sc_Y = StandardScaler()
X = sc_X.fit_transform(X)
y = sc_Y.fit_transform(y)

y=y.ravel()
y

array([0., 0., 0., ..., 0., 0., 0.])

In [102]:
regressor = SVR(kernel = 'rbf')
regressor.fit(X, y)

SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [103]:
def predict(df, regressor):
    df = handle_non_numerical_data(df)
    df = fill_na(df)
    
    return regressor.predict(df)

predict(test, regressor)

array([0., 0., 0., ..., 0., 0., 0.])